<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>LangChain</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from operator import itemgetter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch

import openai
from openai import OpenAI

import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results


import langchain
from langchain.chains import create_sql_query_chain
from langchain.tools import DuckDuckGoSearchRun

import langchain_openai
from langchain_openai import ChatOpenAI

import langchain_anthropic
from langchain_anthropic import ChatAnthropic

import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough

import langchain_community
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

Compiler    : Clang 17.0.0 (clang-1700.0.13.3)
OS          : Darwin
Release     : 25.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 59665e09cdb9e15991a6739b876a356cbd24f05a

matplotlib         : 3.10.3
langchain_openai   : 0.3.18
langchain_community: 0.3.24
langchain_anthropic: 0.3.14
langchain_core     : 0.3.62
numpy              : 2.2.5
watermark          : 2.5.0
openai             : 1.78.1
transformers       : 4.52.3
pandas             : 2.2.3
langchain          : 0.3.25
torch              : 2.7.0



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')

# OpenAI

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key. Then we are ready to instantiate the client

In [4]:
client = OpenAI()

In [5]:
print("\n".join([model.id for model in client.models.list().data]))

gpt-4-0613
gpt-4
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
computer-use-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
computer-use-preview-2025-03-11
gpt-4o-search-preview-2025-03-11
gpt-4o-sea

In [6]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

In [7]:
print(response)

ChatCompletion(id='chatcmpl-D0SRzbrk7MdxURiwz528cyMJazxAD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's most well-known weakness is kryptonite, a mineral from his home planet of Krypton. Exposure to kryptonite weakens Superman and can be lethal with prolonged exposure. There are different types of kryptonite, each having distinct effects on him; the most common is green kryptonite, which drains his powers and can potentially kill him. Other weaknesses include red solar radiation, which strips him of his powers by mimicking the sun of Krypton, and magic, which can affect him like it would any ordinary person.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1769001735, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_deacdd5f6f', usage=CompletionUsage(completion_tokens=104, prompt_tokens=13, total_tokens=117, 

In [8]:
print(response.choices[0].message.content)

Superman's most well-known weakness is kryptonite, a mineral from his home planet of Krypton. Exposure to kryptonite weakens Superman and can be lethal with prolonged exposure. There are different types of kryptonite, each having distinct effects on him; the most common is green kryptonite, which drains his powers and can potentially kill him. Other weaknesses include red solar radiation, which strips him of his powers by mimicking the sun of Krypton, and magic, which can affect him like it would any ordinary person.


# LangChain

We instantiate the LangChain interface for OpenAI

In [9]:
model = ChatOpenAI(model="gpt-4o")

In [10]:
messages = [
    SystemMessage(content="What was Superman's weakness?"),
]

output = model.invoke(messages)
print(output)

content="Superman's primary weakness is Kryptonite, a mineral from his home planet, Krypton. Exposure to Kryptonite can weaken Superman, stripping him of his powers, and prolonged exposure can be fatal. Kryptonite comes in various forms, with green Kryptonite being the most common and well-known version that affects Superman.\n\nIn addition to Kryptonite, Superman is also vulnerable to red solar radiation, which can depower him by mimicking the radiation of his home planet's red sun. Magic is another vulnerability; Superman's invulnerability does not protect him against magical forces and spells. While not a traditional weakness, Superman's strong ethical code and his dedication to protecting others can sometimes be used against him by his enemies." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 141, 'prompt_tokens': 13, 'total_tokens': 154, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens':

In [11]:
output.response_metadata["token_usage"]

{'completion_tokens': 141,
 'prompt_tokens': 13,
 'total_tokens': 154,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

In [12]:
parser = StrOutputParser()

In [13]:
result = model.invoke(messages)

In [14]:
parser.invoke(result)

"Superman's primary weakness is kryptonite, a radioactive fragment from his home planet of Krypton. Exposure to kryptonite weakens him, drains his powers, and can ultimately be lethal if he is in close contact with it for an extended period. Additionally, Superman is also vulnerable to magic and red solar radiation, which can neutralize his abilities. Beyond these, he has the same vulnerabilities as any human being, such as emotional and moral challenges, which can affect him deeply."

Let us create our first chain. Stages of the chain are conencted with the pipe '|' character

In [15]:
chain = model | parser

Now whenver we call __invoke()__ on the chain, it automatically runs all the steps

In [16]:
chain.invoke(messages)

"Superman's primary weakness is kryptonite, a mineral from his home planet of Krypton. When he is exposed to kryptonite, it weakens him significantly, stripping him of his powers and potentially leading to severe illness or even death with prolonged exposure. Different forms of kryptonite can have varying effects, but the most commonly depicted is green kryptonite. Aside from kryptonite, Superman's powers can be diminished under a red sun, which mimics the conditions of Krypton's original solar environment."

We can also create templates for our prompts, following conventions similar to the Jinja templating system

In [17]:
system_template = "Translate the following text into {language}:"

And we can combine multiple messages into a single template

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [
     ("system", system_template), 
     ("user", "{text}")
    ]
)

To instantiate the prompt, we must provide the correct fields

In [19]:
result = prompt_template.invoke(
    {
        "language": "italian", 
        "text": "Be the change that you wish to see in the world."
    }
)

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following text into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Be the change that you wish to see in the world.', additional_kwargs={}, response_metadata={})])

The full interaction is:

In [20]:
result.to_messages()

[SystemMessage(content='Translate the following text into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Be the change that you wish to see in the world.', additional_kwargs={}, response_metadata={})]

In [21]:
chain = prompt_template | model | parser

In [22]:
chain.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

'Sei addestrato sui dati fino a ottobre 2023.'

# Anthropic

In [23]:
?ChatAnthropic

Init signature:
ChatAnthropic(
    *args: Any,
    name: Optional[str] = None,
    cache: Union[langchain_core.caches.BaseCache, bool, NoneType] = None,
    verbose: bool = <factory>,
    callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    tags: Optional[list[str]] = None,
    metadata: Optional[dict[str, Any]] = None,
    custom_get_token_ids: Optional[Callable[[str], list[int]]] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    rate_limiter: Optional[langchain_core.rate_limiters.BaseRateLimiter] = None,
    disable_streaming: Union[bool, Literal['tool_calling']] = False,
    model_name: str,
    max_tokens_to_sample: int = 1024,
    temperature: Optional[float] = None,
    top_k: Optional[int] = None,
    top_p: Optional[float] = None,
    timeout: Optional[float] = None,
    max_retries: int = 2,
    stop: Optional[list[str]] = None,
    

In [24]:
model_a = ChatAnthropic(model="claude-haiku-4-5-20251001")

In [25]:
model_a.invoke("What is Superman's weakness?")

AIMessage(content="Superman's primary weakness is **kryptonite**, radioactive fragments from his home planet Krypton. Exposure to it weakens or can kill him.\n\nHe also has other vulnerabilities:\n\n- **Magic** - he has no special resistance to magical attacks\n- **Red sun radiation** - rays from a red sun (unlike Earth's yellow sun that powers him) drain his abilities\n- **Psychological weaknesses** - his love for people he cares about can be exploited; his strong moral code sometimes limits his actions\n\nKryptonite remains his most famous and commonly referenced weakness across Superman stories.", additional_kwargs={}, response_metadata={'id': 'msg_01QzWrkYbVQE14gedZcfRasG', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 13, 'output_tokens': 133, 'server_tool_use': None, 'service_tier': 'standard', 'cache_creation': {'ephemeral_5m_input_tokens': 0, 'ephe

In [26]:
chain_a = prompt_template | model_a | parser

In [27]:
model_a

ChatAnthropic(model='claude-haiku-4-5-20251001', anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={})

In [28]:
chain_a.invoke(
    {
        "language": "italian", 
        "text": "Be the change that you wish to see in the world."
    }
)

'# Sii il cambiamento che desideri vedere nel mondo.'

# Message History

In [29]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [30]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [31]:
config = {"configurable": {"session_id": "abc2"}}

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bruno")],
    config=config,
)

response.content

'Hello Bruno! How can I assist you today?'

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You just mentioned that your name is Bruno. How can I help you further?'

In [34]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bruno", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bruno! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-D0SSGQZST5oZUF2SLJrRGytYpEgR4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c3889f24-3e93-40bc-a8ba-04789a1d3e1b-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content="What's my nam

In [35]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I can't determine your name based on this information."

In [36]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Bruno. If there's anything else you'd like to know or discuss, feel free to let me know!"

In [37]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [38]:
chain = prompt | model | parser

In [39]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response

'Hello, Bob! How can I assist you today?'

In [40]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [41]:
config = {"configurable": {"session_id": "abc5"}}

In [42]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response

'Hello, Jim! How can I assist you today?'

# Database Integration

In [43]:
db = SQLDatabase.from_uri("sqlite:///data/Northwind_small.sqlite")

In [44]:
print(db.dialect)

sqlite


In [45]:
print(db.get_usable_table_names())

['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


In [46]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [47]:
write_query = create_sql_query_chain(llm, db)

In [48]:
response = write_query.invoke({"question": "How many customers are there"}) 
response

'SELECT COUNT("Id") AS TotalCustomers\nFROM Customer;'

In [49]:
db.run(response)

'[(91,)]'

In [50]:
execute_query = QuerySQLDataBaseTool(db=db)

/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_3694/3528740886.py:1: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [51]:
sql_chain = write_query | execute_query

In [52]:
sql_chain.invoke({"question": "How many employees are there"})

'[(9,)]'

In [53]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are a total of 9 employees.'

In [54]:
RunnablePassthrough.assign(query=write_query).invoke({"question": "How many employees are there"})

{'question': 'How many employees are there',
 'query': 'SELECT COUNT("Id") AS TotalEmployees\nFROM Employee'}

In [55]:
RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query")).invoke({"question": "How many employees are there"})

{'question': 'How many employees are there',
 'query': 'SELECT COUNT("Id") AS TotalEmployees FROM Employee;',
 'result': 'SELECT COUNT("Id") AS TotalEmployees FROM Employee;'}

In [56]:
search = DuckDuckGoSearchRun()
search.run("When will the next solar eclipse be?")

"Future Eclipses Get Ready for These Upcoming Eclipses! Solar Eclipses ... The date listed for each eclipse is the local date where the eclipse occurs. Lunar Eclipses ... Eclipse News More NASA News When's the next total solar eclipse? If seeing these amazing celestial spectacles makes you eager to see the next one, here is a short list of dates for the U.S. and Canada. For travelers, we've also highlighted three upcoming eclipses across the globe.. After the 2024 total solar eclipse, astronomy lovers are eager to know when the next extraterrestrial event will be visible in the U.S. Here is the schedule for the upcoming solar eclipses. The path of the next total solar eclipse to cross Alabama on August 12, 2045 From 1900 to 2100, the state of Alabama will have recorded a total of 87 solar eclipses, two of which are annular eclipses and four of which are total eclipses. The two annular solar eclipses occurred on April 7, 1940, and May 30, 1984, respectively. One total solar eclipse occu

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>